# Flight Delay Prediction

## Data Loading and Preprocessing

In [ ]:
import cudf
import cupy as cp

In [ ]:
# Load the cleaned flight data as a sample
flights_df = cudf.read_parquet("https://github.com/JakeMalis/DS-3000-Final/raw/refs/heads/main/cleaned_flights.parquet").sample(frac=0.25)

In [ ]:
# Extract the hour from the 'DATE' column and create a new column 'DEPARTURE_HOUR'
flights_df['DEPARTURE_HOUR'] = flights_df['DATE'].dt.hour

In [ ]:
# Replace missing values in 'DAILY_SNOWFALL' with 0
flights_df['DAILY_SNOWFALL'] = flights_df['DAILY_SNOWFALL'].fillna(0)

In [ ]:
# Remove flights missing an ARRIVAL_DELAY value
flights_df = flights_df.dropna(subset=['ARRIVAL_DELAY'])

## scikit-learn Models

In [ ]:
from cuml.metrics import mean_squared_error, r2_score
from cuml.linear_model import LinearRegression, Ridge, Lasso
from cuml.ensemble import RandomForestRegressor

In [ ]:
# Standardize numeric columns for sklearn models
for col in ['MONTH', 'DEPARTURE_HOUR', 'DAY_OF_WEEK', 'DISTANCE']:
    col_mean = flights_df[col].mean()
    col_std  = flights_df[col].std()
    flights_df[col] = (flights_df[col] - col_mean) / col_std

In [ ]:
# Extract features for sklearn models
numeric_feats = flights_df[['MONTH', 'DEPARTURE_HOUR', 'DAY_OF_WEEK', 'DISTANCE', 'DAILY_SNOWFALL']].astype(cp.float32).values
categorical_feats = cudf.get_dummies(flights_df[['AIRLINE', 'origin_airport/AIRPORT', 'destination_airport/AIRPORT']]).values.astype(cp.float32)
X = cp.hstack([numeric_feats, categorical_feats])
y = flights_df['ARRIVAL_DELAY'].values

In [ ]:
split = int(0.8 * X.shape[0])
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
# Quadratic Regression
quadratic_model = LinearRegression(fit_intercept=True, normalize=True, algorithm='eig')
quadratic_model.fit(X_train, y_train)

# Get predictions
quadratic_predictions = quadratic_model.predict(X_test)

# Print the MSE, R2, and RMSE
quadratic_r2 = r2_score(y_test.get(), quadratic_predictions)
quadratic_mse = mean_squared_error(y_test.get(), quadratic_predictions)
quadratic_rmse = cp.sqrt(quadratic_mse)
print(f"Quadratic test R2: {quadratic_r2}, MSE: {quadratic_mse}, RMSE: {quadratic_rmse}")

In [ ]:
# Lasso Regression
lasso_model = Lasso(verbose=6)
lasso_model.fit(X_train, y_train)

# Get predictions
lasso_predictions = lasso_model.predict(X_test)

# Print the MSE, R2, and RMSE
lasso_r2 = r2_score(y_test.get(), lasso_predictions)
lasso_mse = mean_squared_error(y_test.get(), lasso_predictions)
lasso_rmse = cp.sqrt(lasso_mse)
print(f"Lasso test R2: {lasso_r2}, MSE: {lasso_mse}, RMSE: {lasso_rmse}")

In [ ]:
# Ridge Regression
ridge_model = Ridge(verbose=6)
ridge_model.fit(X_train, y_train)

# Get predictions
ridge_predictions = ridge_model.predict(X_test)

# Print the MSE, R2, and RMSE
ridge_r2 = r2_score(y_test.get(), ridge_predictions)
ridge_mse = mean_squared_error(y_test.get(), ridge_predictions)
ridge_rmse = cp.sqrt(ridge_mse)
print(f"Ridge test R2: {ridge_r2}, MSE: {ridge_mse}, RMSE: {ridge_rmse}")

In [ ]:
# Random Forest without cross-validation
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_streams=1, verbose=6)
rf_model.fit(X_train, y_train)

# Get predictions
rf_predictions = rf_model.predict(X_test)

# Print the MSE, R2, and RMSE
rf_r2 = r2_score(y_test.get(), rf_predictions)
rf_mse = mean_squared_error(y_test.get(), rf_predictions)
rf_rmse = cp.sqrt(rf_mse)
print(f"RF test R2: {rf_r2}, MSE: {rf_mse}, RMSE: {rf_rmse}")